In [3]:
%matplotlib inline
from keras import *
from keras.layers import *
from keras import regularizers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import LearningRateScheduler, TensorBoard
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
df_train = pd.read_csv("train.csv",header=None,delimiter=',',dtype='float64',names=list(range(4096))).fillna(0)
X_train = df_train.get_values()
df_label = pd.read_csv("train_label.csv",delimiter=',')
Y_train = df_label['category'].get_values()
Y_train = Y_train.reshape(-1,1)
print(X_train.shape)


(113636, 4096)


In [15]:
maxlen = 4096
embed_size = 128
def scheduler(epoch):
    if epoch%1==0 and epoch!=0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr*.8)
        print("lr changed to {}".format(lr*.8))
    return K.get_value(model.optimizer.lr)

lr_decay = LearningRateScheduler(scheduler)

main_input = Input(shape=(maxlen,), dtype='float64',name='main_input')
emb = Embedding(256, embed_size, input_length=maxlen,embeddings_regularizer=regularizers.l2(1e-4))(main_input)
x = Flatten()(emb)

x = Dense(100, activation='linear',kernel_regularizer=regularizers.l2(1e-4))(x)
x = BatchNormalization()(x)
x = ELU()(x)
x = Dropout(0.5)(x)

x = Dense(100, activation='linear',kernel_regularizer=regularizers.l2(1e-4))(x)
x = BatchNormalization()(x)
x = ELU()(x)
x = Dropout(0.5)(x)

x = Dense(100, activation='linear',kernel_regularizer=regularizers.l2(1e-4))(x)
x = BatchNormalization()(x)
x = ELU()(x)
x = Dropout(0.5)(x)

x = Dense(100, activation='linear',kernel_regularizer=regularizers.l2(1e-4))(x)
x = BatchNormalization()(x)
x = ELU()(x)
x = Dropout(0.5)(x)

#num_layers = 4
#for i in range(num_layers):
    #x = Dense(10, activation='linear',kernel_regularizer=regularizers.l2(1e-4))(x)
    #x = BatchNormalization()(x)
    #x = ELU()(x)
    #x = Dropout(0.5)(x)

loss_out = Dense(1, activation='sigmoid',name='loss_out')(x)
model = Model(input=[main_input], output=[loss_out])
optimizer = Adam(lr=0.001)#RMSprop(lr=0.01)
model.compile(optimizer, loss='binary_crossentropy',metrics=['mse', 'accuracy'])
print(model.summary())
history = model.fit(x=X_train, y=Y_train, batch_size=200, epochs=8, verbose=1, callbacks=[lr_decay], validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

x_test = pd.read_csv("test.csv",header=None,delimiter=',',dtype='float64',names=list(range(4096))).fillna(0)
X_test = df_train.get_values()
print(x_test.shape)



/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 4096)              0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 4096, 128)         32768     
_________________________________________________________________
flatten_10 (Flatten)         (None, 524288)            0         
_________________________________________________________________
dense_37 (Dense)             (None, 100)               52428900  
_________________________________________________________________
batch_normalization_37 (Batc (None, 100)               400       
_________________________________________________________________
elu_37 (ELU)                 (None, 100)               0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 100)               0         
__________

In [ ]:
y=model.predict(x_test, batch_size=None, verbose=0, steps=None)
#y[y>=0.5]=1
#y=y.astype(int)
print(y.shape[0])

s=np.arange(y.shape[0]).astype(int)
a= np.zeros((y.shape[0],2)) 
a= np.vstack([s,y.reshape(-1)])
print(a.T)
np.savetxt("out.csv", a.T, delimiter=',', fmt='%i,%f', header="sample_id,malware", comments="")
print("finish")

In [10]:
print(history.history)

{'loss': [0.3982517058034688, 0.235020030365813], 'mean_squared_error': [0.11319923784436023, 0.04613613258422957], 'acc': [0.8469147104291974, 0.9508782388827521]}
